In [1]:
%load_ext autoreload
import abc
import tensorflow as tf
import numpy as np

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec, tensor_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

import gym
from gym import spaces

tf.compat.v1.enable_v2_behavior()

In [30]:
l = tf.keras.layers.Dense(10)
l(tf.ones((1, 5, 5)))

<tf.Tensor: shape=(1, 5, 10), dtype=float32, numpy=
array([[[ 0.2924617 ,  0.50298816,  0.884974  ,  1.6699173 ,
          0.25338107, -0.5293552 , -1.1696597 ,  0.3856331 ,
         -1.7403615 ,  0.14715016],
        [ 0.2924617 ,  0.50298816,  0.884974  ,  1.6699173 ,
          0.25338107, -0.5293552 , -1.1696597 ,  0.3856331 ,
         -1.7403615 ,  0.14715016],
        [ 0.2924617 ,  0.50298816,  0.884974  ,  1.6699173 ,
          0.25338107, -0.5293552 , -1.1696597 ,  0.3856331 ,
         -1.7403615 ,  0.14715016],
        [ 0.2924617 ,  0.50298816,  0.884974  ,  1.6699173 ,
          0.25338107, -0.5293552 , -1.1696597 ,  0.3856331 ,
         -1.7403615 ,  0.14715016],
        [ 0.2924617 ,  0.50298816,  0.884974  ,  1.6699173 ,
          0.25338107, -0.5293552 , -1.1696597 ,  0.3856331 ,
         -1.7403615 ,  0.14715016]]], dtype=float32)>

In [37]:
from tf_agents.networks import encoding_network

input_tensor_spec = {"a": tensor_spec.TensorSpec((2,4), tf.float32),
                     "b": tensor_spec.TensorSpec((2,4), tf.float32)}

preprocessing_layers = {
#     'a': tf.keras.models.Sequential([tf.keras.layers.Conv2D(8, 4),
#                                         tf.keras.layers.Flatten()]),
    "a": tf.keras.layers.Dense(10),
    "b": tf.keras.layers.Dense(10)
}
# preprocessing_combiner = tf.keras.models.Sequential([
#     tf.keras.layers.Concatenate(axis=-1), tf.keras.layers.Flatten()])
preprocessing_combiner = tf.keras.layers.Concatenate(axis=-1)

encoder = encoding_network.EncodingNetwork(
        input_tensor_spec,
        preprocessing_layers=preprocessing_layers,
        preprocessing_combiner=preprocessing_combiner,
#         conv_layer_params=conv_layer_params,
#         fc_layer_params=fc_layer_params,
#         dropout_layer_params=dropout_layer_params,
#         activation_fn=activation_fn,
#         kernel_initializer=kernel_initializer,
#         batch_squash=batch_squash,
#         dtype=dtype
)

state, network_state = encoder({"a": tf.random.uniform((2,4)), "b": tf.random.uniform((2,4))})
state, network_state

(<tf.Tensor: shape=(40,), dtype=float32, numpy=
 array([-5.1287347e-01,  7.2467715e-01,  1.4215332e-01, -4.6144518e-01,
        -5.1920116e-04, -5.7463795e-01, -5.0784940e-01, -3.7329322e-01,
        -1.9996470e-01,  5.0148897e-02,  7.5409435e-02, -2.8834799e-01,
        -4.5258304e-01,  4.2772305e-01, -1.2365946e-01, -3.4159163e-01,
        -3.5489935e-01,  4.2259941e-01, -9.5947444e-02,  4.1757652e-01,
        -3.3949471e-01,  6.8841767e-01,  1.0388622e-01, -6.1435246e-01,
        -2.1779546e-01, -5.5625534e-01, -5.1855332e-01, -4.2009071e-01,
        -2.8448351e-02,  9.0880163e-02, -1.3275978e-01, -5.6241113e-01,
        -1.4250308e-01, -4.4859648e-02, -2.9624620e-01,  3.9942724e-01,
        -5.2600259e-01,  5.4596141e-02, -1.0717499e-01,  1.9253843e-01],
       dtype=float32)>, ())

In [3]:
%autoreload 2
%cd /tf/agents/my-src

from tf_agents.environments import parallel_py_environment
from tf_agents.agents.ppo import ppo_agent
from tf_agents.networks import actor_distribution_network, value_network
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.drivers import dynamic_episode_driver

from canvas_env import CanvasEnv, DummyActorNet, DummyValueNet

num_parallel_environments = 1
collect_episodes_per_iteration = 30

tf_env = tf_py_environment.TFPyEnvironment(
    parallel_py_environment.ParallelPyEnvironment(
        [lambda: tf_py_environment.TFPyEnvironment(
            suite_gym.wrap_env(CanvasEnv()))] * num_parallel_environments))

print(tf_env.time_step_spec())
print(tf_env.action_spec())
print(tf_env.observation_spec())

actor_net = actor_distribution_network.ActorDistributionNetwork(
                tf_env.observation_spec(),
                tf_env.action_spec())
value_net = value_network.ValueNetwork(tf_env.observation_spec())

tf_agent = ppo_agent.PPOAgent(
    tf_env.time_step_spec(),
    tf_env.action_spec(),
    tf.compat.v1.train.AdamOptimizer(),
    actor_net=actor_net,
    value_net=value_net,
    normalize_observations=False,
    use_gae=False)

tf_agent.initialize()

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    tf_agent.collect_data_spec,
    batch_size=num_parallel_environments)
collect_driver = dynamic_episode_driver.DynamicEpisodeDriver(
    tf_env,
    tf_agent.collect_policy,
    observers=[replay_buffer.add_batch],
    num_episodes=collect_episodes_per_iteration)
collect_driver.run()

trajectories = replay_buffer.gather_all()
# print(trajectories)
loss = tf_agent.train(trajectories)

print(loss)

/tf/agents/my-src
TimeStep(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), observation=BoundedTensorSpec(shape=(8,), dtype=tf.float32, name='observation', minimum=array(-10., dtype=float32), maximum=array(10., dtype=float32)))
(BoundedTensorSpec(shape=(), dtype=tf.int64, name='action/tuple_0', minimum=array(0), maximum=array(1)), BoundedTensorSpec(shape=(), dtype=tf.int64, name='action/tuple_1', minimum=array(0), maximum=array(4)), BoundedTensorSpec(shape=(), dtype=tf.int64, name='action/tuple_2', minimum=array(0), maximum=array(4)))
BoundedTensorSpec(shape=(8,), dtype=tf.float32, name='observation', minimum=array(-10., dtype=float32), maximum=array(10., dtype=float32))
(array(0), array(4), array(1))
(array(0), array(2), array(0))
(array(1), array(2), array(3))
(array(

(array(0), array(1), array(3))
(array(1), array(2), array(2))
(array(0), array(4), array(0))
(array(1), array(4), array(0))
(array(1), array(4), array(3))
(array(1), array(1), array(1))
(array(0), array(0), array(3))
(array(1), array(2), array(3))
(array(1), array(1), array(4))
(array(1), array(4), array(4))
(array(0), array(1), array(3))
(array(1), array(2), array(1))
(array(0), array(3), array(2))
(array(1), array(3), array(3))
(array(1), array(3), array(3))
(array(1), array(1), array(0))
(array(1), array(4), array(4))
(array(0), array(4), array(2))
(array(0), array(0), array(3))
(array(1), array(1), array(0))
(array(0), array(3), array(1))
(array(0), array(1), array(3))
(array(0), array(4), array(3))
(array(1), array(3), array(2))
(array(1), array(2), array(4))
(array(0), array(4), array(4))
(array(1), array(3), array(2))
(array(0), array(1), array(1))
(array(1), array(3), array(4))
(array(1), array(1), array(2))
(array(1), array(4), array(2))
(array(0), array(2), array(1))
(array(0

In [1]:
import tensorflow as tf
from tf_agents.trajectories import time_step as ts
from tf_agents.specs import tensor_spec
from tf_agents.policies import random_tf_policy

action_spec = tensor_spec.BoundedTensorSpec(
    (2,), tf.float32, minimum=-1, maximum=3)
input_tensor_spec = tensor_spec.TensorSpec((2,), tf.float32)
time_step_spec = ts.time_step_spec(input_tensor_spec)

my_random_tf_policy = random_tf_policy.RandomTFPolicy(
    action_spec=action_spec, time_step_spec=time_step_spec)
observation = tf.ones(time_step_spec.observation.shape)
time_step = ts.restart(observation)
action_step = my_random_tf_policy.action(time_step)

print('Action:')
print(action_step.action)

Action:
tf.Tensor([1.9907088  0.25149918], shape=(2,), dtype=float32)


In [2]:
class SimpleCorridor(gym.Env):
    """Example of a custom env in which you have to walk down a corridor.

    You can configure the length of the corridor via the env config."""

    def __init__(self):
        self.end_pos = 5
        self.cur_pos = 0
        # self.action_space = spaces.Discrete(2)
        self.action_space = spaces.Tuple([spaces.Discrete(2), spaces.Discrete(4)])
        # self.observation_space = Box(0.0, self.end_pos, shape=(1,), dtype=np.float32)
        self.observation_space = spaces.Dict(
            {
                "a": spaces.Box(0.0, self.end_pos, shape=(1,), dtype=np.float32),
                "b": spaces.Box(0.0, self.end_pos, shape=(1,), dtype=np.float32),
            }
        )

    def reset(self):
        self.cur_pos = 0
        # return [self.cur_pos]
        return {"a": [self.cur_pos], "b": [self.cur_pos]}

    def step(self, action):
        """
        Returns:
            obs
            reward
            done
            log
        """
        # assert action in [0, 1], action
        if action[0] == 0 and self.cur_pos > 0:
            self.cur_pos -= 1
        elif action[0] == 1:
            self.cur_pos += 1
        done = self.cur_pos >= self.end_pos
        return {"a": [self.cur_pos], "b": [self.cur_pos]}, 1 if done else 0, done, {}


    
env = SimpleCorridor()
env = suite_gym.wrap_env(env)
tf_env = tf_py_environment.TFPyEnvironment(env)
tf_env.observation_spec()

OrderedDict([('a',
              BoundedTensorSpec(shape=(1,), dtype=tf.float32, name='observation/a', minimum=array(0., dtype=float32), maximum=array(5., dtype=float32))),
             ('b',
              BoundedTensorSpec(shape=(1,), dtype=tf.float32, name='observation/b', minimum=array(0., dtype=float32), maximum=array(5., dtype=float32)))])

In [ ]:
from tf_agents.agents.ppo import ppo_agent
from tf_agents.drivers import dynamic_episode_driver
from tf_agents.environments import parallel_py_environment
from tf_agents.environments import suite_mujoco
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.networks import actor_distribution_rnn_network
from tf_agents.networks import value_network
from tf_agents.networks import value_rnn_network
from tf_agents.policies import policy_saver
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.utils import common

actor_fc_layers=(200, 100)
value_fc_layers=(200, 100)

global_step = tf.compat.v1.train.get_or_create_global_step()
with tf.compat.v2.summary.record_if(
  lambda: tf.math.equal(global_step % summary_interval, 0)):
    tf.compat.v1.set_random_seed(random_seed)
    eval_tf_env = tf_py_environment.TFPyEnvironment(env_load_fn(env_name))
    tf_env = tf_py_environment.TFPyEnvironment(
        parallel_py_environment.ParallelPyEnvironment(
            [lambda: env_load_fn(env_name)] * num_parallel_environments))
    optimizer = tf.compat.v1.train.AdamOptimizer()

    actor_net = actor_distribution_network.ActorDistributionNetwork(
          tf_env.observation_spec(),
          tf_env.action_spec(),
          fc_layer_params=actor_fc_layers)
    value_net = value_network.ValueNetwork(
          tf_env.observation_spec(), fc_layer_params=value_fc_layers)

    tf_agent = ppo_agent.PPOAgent(
        tf_env.time_step_spec(),
        tf_env.action_spec(),
        optimizer,
        actor_net=actor_net,
        value_net=value_net,
        num_epochs=100,
#         debug_summaries=debug_summaries,
#         summarize_grads_and_vars=summarize_grads_and_vars,
#         train_step_counter=global_step
    )
    tf_agent.initialize()

    environment_steps_metric = tf_metrics.EnvironmentSteps()
    step_metrics = [
        tf_metrics.NumberOfEpisodes(),
        environment_steps_metric,
    ]

    train_metrics = step_metrics + [
        tf_metrics.AverageReturnMetric(
            batch_size=num_parallel_environments),
        tf_metrics.AverageEpisodeLengthMetric(
            batch_size=num_parallel_environments),
    ]

    eval_policy = tf_agent.policy
    collect_policy = tf_agent.collect_policy

    replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
        tf_agent.collect_data_spec,
        batch_size=num_parallel_environments,
        max_length=replay_buffer_capacity)

    train_checkpointer = common.Checkpointer(
        ckpt_dir=train_dir,
        agent=tf_agent,
        global_step=global_step,
        metrics=metric_utils.MetricsGroup(train_metrics, 'train_metrics'))
    policy_checkpointer = common.Checkpointer(
        ckpt_dir=os.path.join(train_dir, 'policy'),
        policy=eval_policy,
        global_step=global_step)
    saved_model = policy_saver.PolicySaver(
        eval_policy, train_step=global_step)

    train_checkpointer.initialize_or_restore()

    collect_driver = dynamic_episode_driver.DynamicEpisodeDriver(
        tf_env,
        collect_policy,
        observers=[replay_buffer.add_batch] + train_metrics,
        num_episodes=collect_episodes_per_iteration)

    def train_step():
      trajectories = replay_buffer.gather_all()
      return tf_agent.train(experience=trajectories)

    if use_tf_functions:
      # TODO(b/123828980): Enable once the cause for slowdown was identified.
      collect_driver.run = common.function(collect_driver.run, autograph=False)
      tf_agent.train = common.function(tf_agent.train, autograph=False)
      train_step = common.function(train_step)

    collect_time = 0
    train_time = 0
    timed_at_step = global_step.numpy()

    while environment_steps_metric.result() < num_environment_steps:
      global_step_val = global_step.numpy()
      if global_step_val % eval_interval == 0:
        metric_utils.eager_compute(
            eval_metrics,
            eval_tf_env,
            eval_policy,
            num_episodes=num_eval_episodes,
            train_step=global_step,
            summary_writer=eval_summary_writer,
            summary_prefix='Metrics',
        )

      start_time = time.time()
      collect_driver.run()
      collect_time += time.time() - start_time

      start_time = time.time()
      total_loss, _ = train_step()
      replay_buffer.clear()
      train_time += time.time() - start_time

      for train_metric in train_metrics:
        train_metric.tf_summaries(
            train_step=global_step, step_metrics=step_metrics)

      if global_step_val % log_interval == 0:
        logging.info('step = %d, loss = %f', global_step_val, total_loss)
        steps_per_sec = (
            (global_step_val - timed_at_step) / (collect_time + train_time))
        logging.info('%.3f steps/sec', steps_per_sec)
        logging.info('collect_time = {}, train_time = {}'.format(
            collect_time, train_time))
        with tf.compat.v2.summary.record_if(True):
          tf.compat.v2.summary.scalar(
              name='global_steps_per_sec', data=steps_per_sec, step=global_step)

        if global_step_val % train_checkpoint_interval == 0:
          train_checkpointer.save(global_step=global_step_val)

        if global_step_val % policy_checkpoint_interval == 0:
          policy_checkpointer.save(global_step=global_step_val)
          saved_model_path = os.path.join(
              saved_model_dir, 'policy_' + ('%d' % global_step_val).zfill(9))
          saved_model.save(saved_model_path)

        timed_at_step = global_step_val
        collect_time = 0
        train_time = 0

    # One final eval before exiting.
    metric_utils.eager_compute(
        eval_metrics,
        eval_tf_env,
        eval_policy,
        num_episodes=num_eval_episodes,
        train_step=global_step,
        summary_writer=eval_summary_writer,
        summary_prefix='Metrics',
    )
